In [63]:
import tensorflow as tf
from tensorflow import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
pd.options.display.max_rows = 10000
#np.set_printoptions(threshold=np.inf)
pd.options.mode.chained_assignment = None

In [64]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
def prep_data():
  #load axon guidance pd data
  axon_guidance_pd_snps = pd.read_csv('/content/drive/My Drive/SRADATA2/axon_guidance_pd_snps.csv')
  # crop dataframe to only the rsID column
  axon_guidance_pd_snps_rs_ids = axon_guidance_pd_snps.loc[:,"rsID"]

  #load gwas tier1 snp data
  gwas_tier1_snps = pd.read_csv('/content/drive/My Drive/SRADATA2/gwas_tier1_snps.csv', dtype=str, keep_default_na=False)

  # take out rows of SNP's with emtpy P-values, rsID's, and ssID's
  gwas_tier1_snps.drop(index=gwas_tier1_snps[gwas_tier1_snps['rsID'] == ""].index, inplace=True)
  gwas_tier1_snps.drop(index=gwas_tier1_snps[gwas_tier1_snps['ssID'] == ""].index, inplace=True)
  gwas_tier1_snps.drop(index=gwas_tier1_snps[gwas_tier1_snps['P value'] == ""].index, inplace=True)
  gwas_tier1_snps['rsID'] = 'rs' + gwas_tier1_snps['rsID'].astype(str)

  # cast the P value and Odds Ratio to float
  gwas_tier1_snps = gwas_tier1_snps.astype({'P value': float, 'Odds Ratio': float})

  # filter for only pd-related SNP's: 20 lowest P-values (less than 0.01 anyways)
  gwas_tier1_snps = gwas_tier1_snps.sort_values(['P value'])
  gwas_tier1_pd_snps = gwas_tier1_snps.head(20)
  
  # crop dataframe to only the rsID column
  gwas_tier1_pd_snps_rs_ids = gwas_tier1_pd_snps.loc[:,"rsID"]


  # create both rsId columns into arrays
  gwas_tier1_pd_snps_rs_ids = gwas_tier1_pd_snps_rs_ids.to_numpy()
  axon_guidance_pd_snps_rs_ids = axon_guidance_pd_snps_rs_ids.to_numpy()
  
  # now, we are combining both rsIds and eliminating the duplicates
  pd_rs_ids = np.concatenate((axon_guidance_pd_snps_rs_ids, gwas_tier1_pd_snps_rs_ids))
  pd_rs_ids = np.unique(pd_rs_ids)
  np.savetxt("/content/drive/My Drive/SRADATA2/pd_rs_ids.csv", pd_rs_ids, delimiter=",", fmt='%s')

  return pd_rs_ids


In [66]:
pd_rs_ids = prep_data()


In [67]:
def load_patient_data():
  
  patient_files = ["CHB.hmap", "CEU.hmap", "GIH.hmap", "CHD.hmap", "ASW.hmap", "JPT.hmap", "LWK.hmap", "MEX.hmap", "MKK.hmap", "TSI.hmap", "YRI.hmap"]
  patient_data_columns = np.insert(pd_rs_ids, 0, 'patientID')
  patient_data_columns = np.append(patient_data_columns, "riskValue")
  patient_data_columns = np.append(patient_data_columns, "risk")  
  all_patient_data = pd.DataFrame(columns = patient_data_columns)
  all_patient_data = all_patient_data.astype({"patientID": str, "risk": int})
  
  all_patient_data_train = pd.DataFrame(columns = patient_data_columns)
  all_patient_data_test = pd.DataFrame(columns = patient_data_columns)
  
  all_patient_data_train.drop(columns=['patientID', 'riskValue'], inplace=True)
  all_patient_data_test.drop(columns=['patientID', 'riskValue'], inplace=True)

  all_patient_data_train = all_patient_data_train.astype({"risk": int})
  all_patient_data_test = all_patient_data_test.astype({"risk": int})


  for file in patient_files:
    print('Processing file: ' + file)
    patient_data = pd.read_csv('/content/drive/My Drive/SRADATA2/' + file, sep = ' ', dtype=str)
    patient_data.columns = patient_data.columns.str.replace('rs#', 'rsID')
    patient_data = patient_data[patient_data.rsID.isin(pd_rs_ids)]
    patient_data['alleles'] = patient_data['alleles'].str.replace('/','')

    for index in range(11, patient_data.shape[1]):
      patient_data.loc[patient_data.iloc[:, index] == patient_data.alleles, patient_data.columns[index]] = 1
      patient_data.loc[patient_data.iloc[:, index] != 1, patient_data.columns[index]] = 0
    patient_data.drop(patient_data.iloc[:, 1:11], axis = 1, inplace = True)
    patient_data = patient_data.set_index('rsID').T.rename_axis('patientID').reset_index()

    for column in pd_rs_ids:
      if column not in patient_data.columns:
        patient_data[column]=0
    patient_data = patient_data.reindex(sorted(patient_data.columns[0:patient_data.shape[1]]), axis=1) 

    num_snps = patient_data.shape[1]-1
    patient_data["riskValue"] = (patient_data.iloc[:,1:patient_data.shape[1]] > 0).sum(1)/num_snps
    patient_data["risk"] = 0
    # Any patient with more than 30% mutations among PD snps is high risk
    patient_data.loc[patient_data['riskValue'] > 0.3, 'risk'] = 1
    all_patient_data = pd.concat([all_patient_data, patient_data], axis=0)
    patient_pop_file = file.replace('.hmap', '.csv')
    patient_data.drop(columns=['patientID', 'riskValue'], inplace=True)
    patient_data.to_csv('/content/drive/My Drive/SRADATA2/' + patient_pop_file, index=False)
    
    patient_data_zeros = patient_data.loc[patient_data['risk'] == 0]
    threshold_zeros = int(0.4*(patient_data_zeros.shape[0]))

    patient_data_ones = patient_data.loc[patient_data['risk'] == 1]
    threshold_ones = int(0.9*(patient_data_ones.shape[0]))

    patient_data_train_zeros = patient_data_zeros.iloc[0:threshold_zeros, :]
    patient_data_test_zeros = patient_data_zeros.iloc[threshold_zeros:patient_data_zeros.shape[0], :]

    patient_data_train_ones = patient_data_ones.iloc[0:threshold_ones, :]
    patient_data_test_ones = patient_data_ones.iloc[threshold_ones:patient_data_ones.shape[0], :]

    patient_data_train = pd.concat([patient_data_train_zeros, patient_data_train_ones], axis=0)
    patient_data_test = pd.concat([patient_data_test_zeros, patient_data_test_ones], axis=0)

    all_patient_data_train = pd.concat([all_patient_data_train, patient_data_train], axis=0)
    all_patient_data_test = pd.concat([all_patient_data_test, patient_data_test], axis=0)


  all_patient_data.to_csv('/content/drive/My Drive/SRADATA2/all_patient_data.csv', index=False)
  all_patient_data_train.to_csv('/content/drive/My Drive/SRADATA2/all_patient_data_train.csv', index=False)
  all_patient_data_test.to_csv('/content/drive/My Drive/SRADATA2/all_patient_data_test.csv', index=False)


In [68]:
# Uncomment this code to prep patient data and generate train/test csvs - after running once, comment it
#load_patient_data()

In [69]:
# Building Neural Network
def load_data():
    train_dataset = pd.read_csv('/content/drive/My Drive/SRADATA2/all_patient_data_train.csv')
    test_dataset = pd.read_csv('/content/drive/My Drive/SRADATA2/all_patient_data_test.csv')

    train_columns = train_dataset.columns
    train_snps = train_dataset[train_columns[train_columns != 'risk']]
    train_risk = train_dataset['risk']
  
    test_columns = test_dataset.columns
    test_snps = test_dataset[test_columns[test_columns != 'risk']]
    test_risk = test_dataset['risk']

    train_snps = to_categorical(train_snps)
    test_snps = to_categorical(test_snps)

    train_risk = to_categorical(train_risk)
    test_risk = to_categorical(test_risk)

    return train_snps, train_risk, test_snps, test_risk

In [70]:
def build_model(input_shape):

    model = Sequential()
    model.add(Flatten())
    model.add(Dense(2, activation='relu', input_shape=input_shape))
    model.add(Dropout(rate=0.2))
    model.add(Dense(2, activation='softmax'))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [71]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix

In [72]:
def run_model():
  
  # load the train and test data from csv files
  train_snps, train_risk, test_snps, test_risk = load_data()
  
  # build and fit the model
  input_shape=(train_snps.shape[1],2)
  print("RUN MODEL: INPUT SHAPE")
  print(input_shape)
  model = build_model(input_shape)
  model.fit(train_snps, train_risk, epochs=100, verbose=2)
  
  # predict risks on test data and compare to actual risks
  scores = model.evaluate(test_snps, test_risk, verbose=0)
  
  # save the model
  model.save('/content/drive/My Drive/SRADATA2/isha_pd_model.h5')
  
  # print accuracy results
  print(scores)
  print('Accuracy: ' + str(scores[1]))

  # predict probabilities for test set
  y_predict_probs = model.predict(test_snps, verbose=0)
  # predict crisp classes for test set
  y_predict_classes = model.predict_classes(test_snps, verbose=0)
  y_probs = y_predict_probs[:, 0]

  print("PREDICT PROBS")
  print(y_predict_probs)
  print("PREDICT CLASSES")
  print(y_predict_classes)
  print("PROBS")
  print(y_probs)
  print("TEST LABEL DATA")
  print(test_risk[:,1])

  # accuracy: (tp + tn) / (p + n)
  accuracy = accuracy_score(np.asarray(test_risk[:,1]), y_predict_classes)
  print('Accuracy: %f' % accuracy)
  # precision tp / (tp + fp)
  precision = precision_score(np.asarray(test_risk[:,1]), y_predict_classes)
  print('Precision: %f' % precision)
  # recall: tp / (tp + fn)
  recall = recall_score(np.asarray(test_risk[:,1]), y_predict_classes)
  print('Recall: %f' % recall)
  f1 = f1_score(np.asarray(test_risk[:,1]), y_predict_classes)
  print('F1 score: %f' % f1)
  # kappa
  kappa = cohen_kappa_score(np.asarray(test_risk[:,1]), y_predict_classes)
  print('Cohens kappa: %f' % kappa)
  # ROC AUC
  auc = roc_auc_score(np.asarray(test_risk[:,1]), y_predict_classes)
  print('ROC AUC: %f' % auc)
  # confusion matrix
  print('Confusion_matrix:')
  print('                    TrueP FalseN')
  print('                    FalseP TrueN')
  #labels must be specified as [1,0]. Without it, confusion_matrix
  #will report negative on the 1st row then positives on the 2nd row
  #note: predictions are in columns and actual values in rows !!
  matrix = confusion_matrix(np.asarray(test_risk[:,1]), y_predict_classes, labels=[1, 0])
  print(matrix)

  return model

In [73]:
model = run_model()

RUN MODEL: INPUT SHAPE
(41, 2)
Epoch 1/100
18/18 - 0s - loss: 0.7291 - accuracy: 0.5413
Epoch 2/100
18/18 - 0s - loss: 0.7302 - accuracy: 0.5483
Epoch 3/100
18/18 - 0s - loss: 0.7292 - accuracy: 0.5343
Epoch 4/100
18/18 - 0s - loss: 0.7234 - accuracy: 0.5360
Epoch 5/100
18/18 - 0s - loss: 0.7070 - accuracy: 0.5501
Epoch 6/100
18/18 - 0s - loss: 0.7227 - accuracy: 0.5466
Epoch 7/100
18/18 - 0s - loss: 0.7083 - accuracy: 0.5483
Epoch 8/100
18/18 - 0s - loss: 0.7032 - accuracy: 0.5606
Epoch 9/100
18/18 - 0s - loss: 0.6959 - accuracy: 0.5729
Epoch 10/100
18/18 - 0s - loss: 0.6955 - accuracy: 0.5782
Epoch 11/100
18/18 - 0s - loss: 0.6905 - accuracy: 0.5729
Epoch 12/100
18/18 - 0s - loss: 0.6936 - accuracy: 0.5835
Epoch 13/100
18/18 - 0s - loss: 0.6979 - accuracy: 0.5712
Epoch 14/100
18/18 - 0s - loss: 0.6941 - accuracy: 0.5729
Epoch 15/100
18/18 - 0s - loss: 0.6854 - accuracy: 0.6046
Epoch 16/100
18/18 - 0s - loss: 0.6817 - accuracy: 0.5905
Epoch 17/100
18/18 - 0s - loss: 0.6833 - accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [74]:
# FOR FIGURE 1: test on each individual population
def test_population():
  pop_files = ["CHB.csv", "CEU.csv", "GIH.csv", "CHD.csv", "ASW.csv", "JPT.csv", "LWK.csv", "MEX.csv", "MKK.csv", "TSI.csv", "YRI.csv"]
  for file in pop_files:
    print('Processing file: ' + file)
    pop_data = pd.read_csv('/content/drive/My Drive/SRADATA2/' + file, sep = ',', dtype=int)

    pop_x = pop_data.iloc[:, 0:pop_data.shape[1]-1]
    pop_y = pop_data.iloc[:, pop_data.shape[1]-1]

    pop_x = to_categorical(pop_x)
    pop_y = to_categorical(pop_y)

    scores = model.evaluate(pop_x, pop_y, verbose=0)
    print('Accuracy: ' + str(scores[1]))


    # predict probabilities for test set
    y_predict_probs = model.predict(pop_x, verbose=0)
    # predict crisp classes for test set
    y_predict_classes = model.predict_classes(pop_x, verbose=0)
    #print(pop_x)
    pop_y[:,1] = [int(x) for x in pop_y[:,1]]
    #print(np.asarray(pop_y[:,1]))
    #print(test_risk[:,1])
    #print(y_predict_classes)
    #print(y_predict_probs)
    #print(y_probs)

    # reduce to 1d array
    y_probs = y_predict_probs[:, 0]
    #print(y_classes)
    #y_classes = y_predict_classes[:, 0]
    #print(y_predict_probs)
    #print(y_predict_classes)
    #print(y_probs)
    #print(y_classes)

    print ("Classification report:", (classification_report(np.asarray(pop_y[:,1]), y_predict_classes)))
    #print ("F1 micro averaging:",(f1_score(np.asarray(pop_y[:,1]), y_predict_classes, average='micro')))
    #print ("ROC: ",(roc_auc_score(np.asarray(pop_y[:,1]), y_probs)))
    #matrix = multilabel_confusion_matrix((np.asarray(pop_y[:,1], y_predict_classes)))
    #print(matrix)

    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(np.asarray(pop_y[:,1]), y_predict_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(np.asarray(pop_y[:,1]), y_predict_classes, average='micro')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(np.asarray(pop_y[:,1]), y_predict_classes, average='micro')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(np.asarray(pop_y[:,1]), y_predict_classes, average='micro')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(np.asarray(pop_y[:,1]), y_predict_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(np.asarray(pop_y[:,1]), y_predict_classes, average='micro')
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print('Confusion_matrix:')
    #print('                    TrueP FalseN')
    #print('                    FalseP TrueN')
    #labels must be specified as [1,0]. Without it, confusion_matrix
    #will report negative on the 1st row then positives on the 2nd row
    #note: predictions are in columns and actual values in rows !!
    matrix = confusion_matrix(np.asarray(pop_y[:,1]), y_predict_classes, labels=[1, 0])
    print(matrix)

In [75]:
test_population()

Processing file: CHB.csv
Accuracy: 0.9024389982223511
Classification report:               precision    recall  f1-score   support

         0.0       0.92      0.97      0.94        68
         1.0       0.80      0.57      0.67        14

    accuracy                           0.90        82
   macro avg       0.86      0.77      0.80        82
weighted avg       0.90      0.90      0.90        82

Accuracy: 0.902439
Precision: 0.902439
Recall: 0.902439
F1 score: 0.902439
Cohens kappa: 0.611374
ROC AUC: 0.771008
Confusion_matrix:
[[ 8  6]
 [ 2 66]]
Processing file: CEU.csv


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses

Accuracy: 0.7716049551963806
Classification report:               precision    recall  f1-score   support

         0.0       0.83      0.81      0.82       106
         1.0       0.66      0.70      0.68        56

    accuracy                           0.77       162
   macro avg       0.75      0.75      0.75       162
weighted avg       0.77      0.77      0.77       162

Accuracy: 0.771605
Precision: 0.771605
Recall: 0.771605
F1 score: 0.771605
Cohens kappa: 0.501414
ROC AUC: 0.753875
Confusion_matrix:
[[39 17]
 [20 86]]
Processing file: GIH.csv
Accuracy: 0.7951807379722595


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses

Classification report:               precision    recall  f1-score   support

         0.0       0.83      0.90      0.86        60
         1.0       0.67      0.52      0.59        23

    accuracy                           0.80        83
   macro avg       0.75      0.71      0.72        83
weighted avg       0.79      0.80      0.79        83

Accuracy: 0.795181
Precision: 0.795181
Recall: 0.795181
F1 score: 0.795181
Cohens kappa: 0.452039
ROC AUC: 0.710870
Confusion_matrix:
[[12 11]
 [ 6 54]]
Processing file: CHD.csv
Accuracy: 0.8857142925262451
Classification report:               precision    recall  f1-score   support

         0.0       0.91      0.95      0.93        55
         1.0       0.77      0.67      0.71        15

    accuracy                           0.89        70
   macro avg       0.84      0.81      0.82        70
weighted avg       0.88      0.89      0.88        70

Accuracy: 0.885714
Precision: 0.885714
Recall: 0.885714
F1 score: 0.885714
Cohens kappa: 0.64

...

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses

Accuracy: 0.8028169274330139
Classification report:               precision    recall  f1-score   support

         0.0       0.87      0.87      0.87        55
         1.0       0.56      0.56      0.56        16

    accuracy                           0.80        71
   macro avg       0.72      0.72      0.72        71
weighted avg       0.80      0.80      0.80        71

Accuracy: 0.802817
Precision: 0.802817
Recall: 0.802817
F1 score: 0.802817
Cohens kappa: 0.435227
ROC AUC: 0.717614
Confusion_matrix:
[[ 9  7]
 [ 7 48]]
Processing file: MKK.csv
Accuracy: 0.8011695742607117


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses

Classification report:               precision    recall  f1-score   support

         0.0       0.85      0.90      0.87       128
         1.0       0.63      0.51      0.56        43

    accuracy                           0.80       171
   macro avg       0.74      0.71      0.72       171
weighted avg       0.79      0.80      0.79       171

Accuracy: 0.801170
Precision: 0.801170
Recall: 0.801170
F1 score: 0.801170
Cohens kappa: 0.437064
ROC AUC: 0.705033
Confusion_matrix:
[[ 22  21]
 [ 13 115]]
Processing file: TSI.csv
Accuracy: 0.8051947951316833
Classification report:               precision    recall  f1-score   support

         0.0       0.88      0.86      0.87        57
         1.0       0.62      0.65      0.63        20

    accuracy                           0.81        77
   macro avg       0.75      0.75      0.75        77
weighted avg       0.81      0.81      0.81        77

Accuracy: 0.805195
Precision: 0.805195
Recall: 0.805195
F1 score: 0.805195
Cohens kappa: 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
